In [27]:
import os
import sys
import time
import boto3
import logging
import pandas as pd
import openmatrix as omx
from zipfile import ZipFile

s3_client = boto3.client('s3')

logging.basicConfig(
    stream=sys.stdout, level=logging.INFO,
    format='%(asctime)s %(name)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)


os.chdir('../')
import kpi

In [ ]:
scenario = '01_base_000'
hh_fpath = os.path.join('/Users/juandavidcaicedocastro/tmp', scenario,'final_households.csv')
pp_fpath = os.path.join('/Users/juandavidcaicedocastro/tmp', scenario,'final_persons.csv')
tt_fpath = os.path.join('/Users/juandavidcaicedocastro/tmp', scenario,'final_trips.csv')
ss_fpath = os.path.join('/Users/juandavidcaicedocastro/tmp', scenario,'skims.omx')

households = pd.read_csv(hh_fpath, index_col = 'household_id')
persons = pd.read_csv(pp_fpath, index_col = 'person_id')
trips = pd.read_csv(tt_fpath, index_col = 'trip_id', dtype = {'origin':int, 'destination':int})
skims = omx.open_file(ss_fpath, 'r')

In [9]:
mapping = kpi.read_yaml('/Users/juandavidcaicedocastro/Dropbox/01_berkeley/22_UrbanSim/github/sensitivy_analysis_carb/mapping.yaml')

In [10]:
%%time
trips_new, persons_new = kpi.add_results_variables(mappings, trips, households, persons, skims)

Mean Persons VOT: 9.330766117704544
Mean TRIP VOT: 11.340948251055327
Trips shape, after merging: (20, 32)
Persons shape after merging: (7512006, 90)
Sum of c_cost: -0.055746031812142445
Sum of c_ivt: -0.47879999999999995
Sum of value of time: 226.81896502110652
Sum of cs: -5371.999653559184
CPU times: user 15.4 s, sys: 16 s, total: 31.4 s
Wall time: 40.1 s


In [14]:
trips_new.columns

Index(['person_id', 'household_id', 'tour_id', 'primary_purpose', 'trip_num',
       'outbound', 'trip_count', 'purpose', 'destination', 'origin',
       'destination_logsum', 'depart', 'trip_mode', 'mode_choice_logsum',
       'dist_miles', 'carb_mode', 'commute', 'period', 'mode_index',
       'travel_time', 'driving_access_mode_index', 'driving_access', 'VMT',
       'c_ivt', 'income', 'lcm_county_id', 'race', 'hispanic', 'value_of_time',
       'income_category', 'c_cost', 'cs'],
      dtype='object')

In [17]:
a = trips_new[['mode_choice_logsum', 'c_ivt', 'income_category','value_of_time' ,'c_cost', 'cs']]

In [18]:
a.groupby('income_category')['cs'].mean()

income_category
low      -215.822580
middle   -385.234115
high     -165.865209
Name: cs, dtype: float64

In [21]:
a['c_cost'].mean()

-0.002787301590607122

In [24]:
a['cs_v2'] = a['mode_choice_logsum']/(- 1* a['c_cost'].mean())

/var/folders/d4/00lkn0rs7j3359dk7430bfy00000gn/T/ipykernel_4847/3254286993.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  a['cs_v2'] = a['mode_choice_logsum']/(- 1* a['c_cost'].mean())


In [31]:
a.groupby('income_category')['cs'].mean()

income_category
low      -215.822580
middle   -385.234115
high     -165.865209
Name: cs, dtype: float64

In [26]:
a.groupby('income_category')['cs_v2'].mean()

income_category
low      -312.848189
middle   -304.612372
high      -79.504452
Name: cs_v2, dtype: float64

In [29]:
a.groupby('income_category')['c_cost'].mean()

income_category
low      -0.005227
middle   -0.002139
high     -0.001213
Name: c_cost, dtype: float64

In [30]:
a.groupby('income_category')['mode_choice_logsum'].mean()

income_category
low      -0.872002
middle   -0.849047
high     -0.221603
Name: mode_choice_logsum, dtype: float64